In [98]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [99]:
hdfsclient = Client("amp-bdg-master.amplab.net", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/home/eecs/akmorrow/ADAM/endive"


In [100]:
executor_mem = "100g"
cores_per_executor = 8
num_executors = 50
EGR1_PATH = "/data/anv/DREAMDATA/aggregated/full_dnase/EGR1"

featurized = "hdfs://amp-spark-master.amp:8020/user/akmorrow/featurized/EGR1_dnase_sampled_test_chr15"

In [ ]:
# this is where the magic happens
results = [] 
featureOutput = "/data/anv/featurized/EGR1_dnase"
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "fakePath",
                           logpath,
                           featureOutput,
                           seed=0,
                           cores_per_executor=cores_per_executor,
                           sample=0.01,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           num_executors=num_executors)
results.append(res)

net.akmorrow13.endive.pipelines.DnaseKernelPipeline
/home/eecs/akmorrow/ADAM/endive
../target/scala-2.10/endive-assembly-0.1.jar
{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'numPartitions': 400, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'aggregatedSequenceOutput': '/data/anv/DREAMDATA/aggregated/full_dnase/EGR1', 'featuresOutput': '/data/anv/featurized/EGR1_dnase', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 256, 'featurizeSample': 0.01, 'readFiltersFromDisk': True}
{'SPARK_EXECUTOR_CORES': '8', 'OMP_NUM_THREADS': '1', 'KEYSTONE_MEM': '100g', 'SPARK_NUM_EXECUTORS': '50'}


In [105]:
# chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# chromosomes.append("chrX")
chromosomes=["chr15"]
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [ ]:
reload(scala_python_endive_wrapper)
results_df = cross_validate(featurized, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            logpath,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001],
                            mixtureWeights=[-1.0],
                            cores_per_executor=8,
                            num_executors=32,
                            regs=[1e-2])

FOLD 0
HOLDING OUT CHROMOSOMES ['chr15']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=0.01
AND MIXTUREWEIGHT=-1.0


In [107]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.001,0.01,[3],[chr15],0.999998,0.995256,1.0,1.0,1.0,1.0,0.077754,0.973423,0.0,0.0,0.0,0.0
